#Lab.02 / IBM3202 – Visualizing and Comparing Molecular Structures in Google Colab using py3Dmol

##Theoretical aspects


As described in **Lab.01**, the PDB file contains information of all the experimental conditions that allowed obtaining a given three-dimensional protein structure (contained in the PDB file), as well as information about the biological features of the protein and the overall features of the crystal structure (cell unit, dimensions, number of monomers, biological assembly, etc.). However, its most relevant body of corresponds to the **coordinates** (i.e. the three-dimensional positions) of the atoms of a structure of a given protein. This is, all PDB files contain the following minimal format:

![alt text](https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_01.png)

In which **ATOM** (or HETATM) indicates that the line contains information of atomic coordinates, a unique number for each atom (a list that is also referred to as index), its atom type, the residue name to which a given atom belongs to, the polypeptide chain in which this residue is located, the position or number of the residue in the primary sequence, and the cartesian atomic coordinates of each atom. Any molecule (protein or not) can be written in this format, as long as we have the cartesian coordinates for each of its atoms.

As you may recall from the lectures, biological structures are gathered by different experimental evidences: **X-ray crystallography**, **Nuclear Magnetic Resonance** or, more recently, **Cryo-Electron microscopy**. This information is far from giving you exact atomic coordinates resolutions, thus measurements of how well defined is this coordinate is also written in this format. Those columns, next to the coordinates, are the occupancy and temperature (or B) factors. The occupancy tells you if the atom is definitely in that position, or if an alternative configuration is also likely. The image in **Figure 1** presents this scenario in a protein structure solved by X-ray crystallography (PDB 6ANE; the same we used in Lab.01), where a given tryptophan residue cannot be univocally assigned to a single configuration (i.e. the electron density is equally fit in both configurations). In this case, this is interpreted as if this residue actually _occupies_ both configurations in the crystal, such that nearly half of the molecules in the crystal lattice take one of the conformers.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_01.gif'/>
<figcaption>FIGURE 1. Occupancies of a tryptophan residue in a crystallographic structure (PDB ID 6ANE). In this case, the same atom has two sets of coordinates, one for each configuration. Hence, the occupancy of each of those coordinate lines is reduced to 0.50. In the general case, when a single atom (line of coordinates) has n possible configurations, its occupancy is reduced to 1/n.</figcaption></center>
</figure>

💡 Hint: The GIF in Figure 1 is the result of comparing the same tryptophan residue after struuctural superposition of two different chains collected in the same X-Ray crystallography experiment. This illustrates the need to carefully inspect a PDB structure before doing any computational study with it, because if there are differences in a side chain that is near the active site that may impact the results of subsequent studies, such as molecular docking 

On the other hand, the **_temperature_ factors** are a measurement of uncertainty in the positions assigned to each atom. In the previous case, occupancy was telling us that we had multiple stable configurations within our structure. Instead, temperature factors (also referred to as isotropic B-values) are measuring how likely it is to see that atom in that particular position. The larger this value, the higher the mobility around our defined position, hence higher the uncertainty. This value depends inversely with resolution: high resolution structures have nicely-detailed atomistic resolution, and we can see with little doubt were each atom should be positioned, while low resolution structures only provide us with coarse features of the molecular ensemble, and coordinates are much more difficult to assign **(Figure 2)**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_02.png'/>
<figcaption>FIGURE 2. Effect of resolution on electron-density maps. In the first case we can clearly see the positions of the heavy atoms, while at lower resolution these positions become uncertain.</figcaption></center>
</figure>

The two parameters just defined are complementary to each other, as the multiple stable conformers arising from uncomplete occupancy are not necessarily equally likely, so mutual information helps you decide which is the most likely **(Figure 3)**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_03.png'/>
<figcaption>FIGURE 3. Coordinate lines for the same atom (CB) in two distinct tryptophan configurations (ATRP, BTRP). As you may notice, the occupancy is half for each, yet the B-factor is lower for the first (A) configuration, hence more likely.</figcaption></center>
</figure>


With this, we have briefly covered the essentials of PDB structures. However, the goodness of a given protein structure deposited in the PDB is present in the webpage of the corresponding PDB identifier, as show in **Figure 4**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_04.png'/>
<figcaption>FIGURE 4. Overall structure quality. This plot depends on the structure and resolution. It ranks from worst (red) to best (blue) structures at the same resolution.</figcaption></center>
</figure>

<p align = "justify">Several parameters allow you to assess the quality of an X-ray solved protein structure. 

>In **Rwork/Rfree**, the former tells you how well fitted are the atomic coordinates to the experimental data, while the latter is a “control” measurement with part of the data that was left out in calculating Rwork. The lower these values are, the better the fit of the solved structure to the electron density. However, be careful if Rwork << Rfree, as it is likely that the data is being overfitted to the experimental electron density.

>The **Clashscore** measures if atoms are unusually close to each other (number of clashes per every 1,000 atoms). 

>The **Ramachandran outliers** measure if there are unusually stretched torsions in the φ- and ψ-dihedral angles of the backbone. 

>Amino acid sidechains also adopt preferred configurations (or rotamers), thus **Sidechain outliers** measure how many uncommon rotamers are present in a given structure. 

>Finally, **RSRZ** is a measurement between the atomic model and the real-space data.

Although the PDB file contains the coordinates of a given structure in text, you have already seen that most solved protein structures are typically shown as visual representations of such atomic coordinates. 

Here, we will use **py3DMol** (python package for dependency-free molecular visualization in iPython notebooks) to visualize and analyze some of these protein structures. It is worth noting that this package will be our fundamental analysis tool for most future tutorials related to work with protein structures.

##Experimental Overview

###Part I. Retrieving and visualizing structures from PDB

<p align = "justify">You can retrieve PDB structures from its website (www.rcsb.org). Alternatively, you can directly use the terminal to download a given PDB file with known accession code as shown below, where XXXX must be replaced by the replaced by the 4-letter PDB code:

```
!wget http://www.rcsb.org/pdb/files/XXXX.pdb.gz 
!gunzip XXXX.pdb.gz
```




In [1]:
 ##Write down your code here to retrieve a protein of your choice!
!wget http://www.rcsb.org/pdb/files/6ANE.pdb.gz 
!gunzip 6ANE.pdb.gz

--2021-02-09 05:38:20--  http://www.rcsb.org/pdb/files/6ANE.pdb.gz
Resolving www.rcsb.org (www.rcsb.org)... 128.6.159.248
Connecting to www.rcsb.org (www.rcsb.org)|128.6.159.248|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.rcsb.org/download/6ANE.pdb.gz [following]
--2021-02-09 05:38:21--  https://files.rcsb.org/download/6ANE.pdb.gz
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.70
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132893 (130K) [application/octet-stream]
Saving to: ‘6ANE.pdb.gz’

6ANE.pdb.gz         100%[===================>] 129.78K  --.-KB/s    in 0.06s   

2021-02-09 05:38:21 (2.20 MB/s) - ‘6ANE.pdb.gz’ saved [132893/132893]



If you open one of such files with a text editor (as we did in Lab.01), you will be able to notice all of the things that we discussed in Lab.01. Basically, you will find plenty of experiment-related information, including the parameters of extraction and solving the crystal structure, followed by the information of the coordinates of each atom that was assigned to a given electron density after the X-ray diffraction and structure solution procedure (noting that these means that atoms with no electron density will not be present in the PDB file).

In [2]:
#Do you want to know how many chains are contained in the PDB
#You just downloaded? grep can help you! 
#(remember to check your directory)
!grep 'COMPND'.*'CHAIN' 6ANE.pdb

#How many residues has each chain? Now we can use awk!
!awk '$1=="ATOM" && $3=="CA" && $5=="A" {print $0}' 6ANE.pdb | wc -l

COMPND   3 CHAIN: A, B, C;                                                      
262


We are going to use [**BioPython**](https://biopython.org/) (a set of freely available tools for biological computation written in Python) to help us to get information from the PDB for each chain. 

As the 6ane.pdb has 3 different chains, here we use the **BioPDB module** from BioPython to split our multichain PDB into three different ones:


In [3]:
#The following line is a usual way of installing python tools/modules
!pip install biopython

#Here, we import Bio.PDB to use it to manipulate PDB files
from Bio.PDB import *
#And we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '6ANE.pdb')

#This will separate each chain into its own PDB file
for chain in structure.get_chains():
    io.set_structure(chain)
    io.save("6ANE_" + chain.get_id() + ".pdb")

     |████████████████████████████████| 2.3MB 4.2MB/s 


/usr/local/lib/python3.6/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6376.
  PDBConstructionWarning,
/usr/local/lib/python3.6/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6531.
  PDBConstructionWarning,
/usr/local/lib/python3.6/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6702.
  PDBConstructionWarning,


To visualize this structure, we will load our PDB file (i.e. 6ane) onto **py3Dmol**, a pyhton implementation of [**3Dmol.js**](http://3dmol.csb.pitt.edu/). You can read the full documentation of 3Dmol.js [here](https://3dmol.csb.pitt.edu/doc/index.html), which is very similar to the one that we are going to use in this tutorial besides some extra stuff here and there.

In [4]:
#Here, write down the code to install py3Dmol
!pip install py3Dmol

In [5]:
#And we already learned how to import these modules, right?
import py3Dmol

##### Let's visualize some PDBs!

In the following code cell we take a line by line look to the different commands needed to visualize two of the chains that you extracted from 6ANE. Please note that the text inside each parenthesis follows the structure of a JSON data structure which is reviewed in detail [here](https://towardsdatascience.com/an-introduction-to-json-c9acb464f43e).

Pay special attention to the following **py3Dmol** classes and what they do: _**addmodel**_, _**setBackground**_ and _**setStyle**_.

💡 Hint: The syntax for loading different structures and properties is really straightforward and self explanatory. Even so, remember to be carefull when opening and closing {} and () and using quotes when needed.


In [16]:
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files of chain B and C
view.addModel(open('6ANE_B.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
#Zooming into all visualized structures 
view.zoomTo()
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style for chain B and C
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
#And we finally visualize the structures using the command below
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

As we can see, we are able to load chains B and C of 6ANE in cartoon representation, but they are far from each other. In case we want to compare the conformations of different residues in these two chains (such as the conformation of the wobbly tryptophan residue mentioned before), we are going to use the **Bio.PDB python module** again, which is implemented in the following script, to: 
1. Superimpose 6ane chain B to chain C
2. Print the RMSD
3. Save the superimposed PDB 

In [11]:
#The following code was created by Anders Steen Christensen
#from the University of Basel and is available at
#https://gist.github.com/andersx/6354971

import Bio.PDB

# Select what residues numbers you wish to align
# and put them in a list
start_id = 1
end_id   = 70
atoms_to_be_aligned = range(start_id, end_id + 1)

# Start the parser
pdb_parser = Bio.PDB.PDBParser(QUIET = True)

# Get the structures
ref_structure = pdb_parser.get_structure("reference", "6ANE_C.pdb")
sample_structure = pdb_parser.get_structure("sample", "6ANE_B.pdb")

# Use the first model in the pdb-files for alignment
# Change the number 0 if you want to align to another structure
ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Make a list of the atoms (in the structures) you wish to align.
# In this case we use CA atoms whose index is in the specified range
ref_atoms = []
sample_atoms = []

# Iterate of all chains in the model in order to find all residues
for ref_chain in ref_model:
  # Iterate of all residues in each model in order to find proper atoms
  for ref_res in ref_chain:
    # Check if residue number ( .get_id() ) is in the list
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Append CA atom to list
      ref_atoms.append(ref_res['CA'])

# Do the same for the sample structure
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Now we initiate the superimposer:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Print RMSD:
print('The calculated RMSD is:')
print (super_imposer.rms)

# Save the aligned version of one of the chains of 6ANE
io = Bio.PDB.PDBIO()
io.set_structure(sample_structure) 
io.save("6ANE_B_aligned.pdb")

The calculated RMSD is:
0.17219476212991125


Now we can load the superimposed structure into **py3dmol** with the code cell show below. Please try to familiarize yourself with these lines by changing the colors of the visualization at will

In [12]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Now we want to inspect the different conformers of Tryptophan 158 between chains B and C of 6ANE. For this, we can select specific residues using _**addStyle**_ classes as shown below

💡 Hint: You can use different color schemes for stick representations. They can be used to control the color of carbon atoms and are usually indicated by a "Carbon" suffix. For example, _**blueCarbon**_ shows all carbon atoms of a stick representation in blue, _**greenCarbon**_ shows all carbon atoms in green, etc.



In [13]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Set a visualization style for chain B
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
#Add a visualization style for residue 158 in chain B
view.addStyle({'chain':'B','resi':158},{'stick':{'colorscheme':'grayCarbon'}})
#Set a visualization style for chain C
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
#Add a visualization style for residue 158 in chain C
view.addStyle({'chain':'C','resi':158},{'stick':{'colorscheme':'skyblueCarbon'}})
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Sometimes we might want to show several residues in our visualization. Instead of writing a line of code per residue, py3Dmol can also work with residue selections:

* A sequence of residues can be given as a range usign square brackets:

> ["158-168"]

* The same can be done for non-sequential residues using comma-separated numbers: 

>[158,168,129,182]

How can we show the catalytic residues of 6ANE, corresponding to residue numbers 133, 179 and 210, and the Tryptophan residue in position 158 with different colors?

**Do it yourself** by appropriately setting up these residues and their color scheme representation in the following lines of code

In [14]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Setting style for chain B
view.setStyle({'chain':'B'},
              {'cartoon': {'color':'purple'}}
             )
#Add your selection residue and choose a color scheme in the next line
view.addStyle({'chain':'B','resi':CHANGE ME},
              {'stick':{'colorscheme':CHANGE ME}})
#Setting style for chain C
view.setStyle({'chain':'C'},
              {'cartoon': {'color':'yellow'}}
             )
#Add a different representation for residue 158 with a different color scheme 
view.addStyle({'chain':'C','resi':CHANGE ME},
              {'stick':{'colorscheme':CHANGE ME}}
             )
view.show()

SyntaxError: ignored

What if I need to select and show residues **around a given distance radius** of Trp158? You can use **_within_** selections in your _addStyle_ classes as shown in the following lines of code

In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Setting styles for chains B and C 
view.setStyle({'chain':'B'},
              {'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},
              {'cartoon': {'color':'yellow'}})
#See residues that are a distance X from the residue 158
view.addStyle({'within':{'distance': 7,
                         'sel':{'resi':158}
                         }
               }
              ,{'stick':{'colorscheme':'greenCarbon'}
                }
              )
#After you made your selection, change colors for the Trp from both chains
view.addStyle({'chain':'B','resi':158},
              {'stick':{'colorscheme':'blueCarbon'}})
view.addStyle({'chain':'C','resi':158},
              {'stick':{'colorscheme':'skyblueCarbon'}})
view.show()

What about other representations, such as the **van der Waals** representation of the atoms? You can display the atom surface using vDW through the _**addSurface**_ class, as shown in the following line of code and used in the cell code below.

```
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, {'chain':'A'})
```

In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'B'},{'cartoon': {'color':'blue'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'skyblue'}})
view.addStyle({'within':{'distance': 7, 'sel':{'resi':158}}},{'stick':{'colorscheme':'grayCarbon'}})
view.addStyle({'chain':'B','resi':158},{'stick':{'colorscheme':'purpleCarbon'}})
view.addStyle({'chain':'C','resi':158},{'stick':{'colorscheme':'greenCarbon'}})
#VDW Surface
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, {'chain':'B'})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'yellow'}, {'chain':'C'})
view.show()

Finally, if you want to identify the residues in your visualization, you can also add **text labels** using the _**addLabel**_ class. However, we first need to know the names of the residues that we would like to label.

The first code cell shows how to easily obtain the name for residue number 158. If all works well, you should get "TRP" as an answer, as you already know.

The second code cell incorporates the _**addLabel**_ class to add a "W158" text label for residue 158.

In [ ]:
#To know the residue type of the selected residues you could use awk! 
#there are more elegant ways to do this but for now...
!awk '$6=="158" && $3=="CA" {print $4}' 6ANE_B.pdb
#Try yourself for residue 210, 179 and 133


In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
view.addStyle({'chain':'B','resi':158},{'stick':{'colorscheme':'blueCarbon'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
view.addStyle({'chain':'C','resi':158},{'stick':{'colorscheme':'skyblueCarbon'}})
#Using this line of code you can specify "The text","Opacity of the label","A selection of residues" 
view.addLabel("W158",{'fontOpacity':1},{'resi':158})

#You can also uncomment the following line to make the model spin
#view.spin("y")

#Visualize your structure 
view.show()

#The following is a small trick to be able to manipulate (rotate, zoom) your
#representation in Google Colab and, once you are happy with the visualization,
#press Enter to print out as an onscreen png image file that you can download
#input()
#view.png()

####A small challenge for you! (Homework)

Try labelling and visualizing the catalytic residues (they have been pointed out in this tutorial) in another chunk of code!

In [ ]:
#Try it yourself here

**This is the end of the second tutorial. Good science!**

#Appendix A. *Load and watch an MD trajectory using the web version of NGLviewer*

In this tutorial we were able to load and visualize static structures with different representation types and colors. However, in the near future we will be performing **Molecular Dynamics Simulations**, for which we will need to visualize **simulation trajectories** (i.e. the atom motions in our simulation).

Unfortunately, for now Google Colab is uncapable of loading such trajectories. Thus, we will employ a web version of **NGLview** for this purpose.

To check how NGLview works, please load one of the files that we will sent you via Google Drive and then explore the selections and representations inside the web version of [NGL viewer](http://nglviewer.org/ngl/)

#Appendix B. *Create a Goodsell like representation in 3D protein Imaging*

You can get beautiful RSCB PDB 101-like representations in a few clicks using the [3D Protein Imaging](https://3dproteinimaging.com/protein-imager) web app.

Try downloading one the PDB chains from the Google Colab session and upload it to 3D Protein Imaging. You can then send us your illustration and we can give you feedback!
<figure><center><img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_05.png'></figure>

#Appendix C. *py3Dmol examples*

Note: This Appendix corresponds to the py3Dmol tutorial in its entirety and available [**here**](https://pypi.org/project/py3Dmol/). Since it has been created to work on IPython/Jupyter, there is a possibility that some sections might not work on Google Colab.


A simple [IPython/Jupyter](http://jupyter.org/) widget to
embed an interactive [3Dmol.js](http://3dmol.csb.pitt.edu) viewer in a notebook.

The widget is completely static, which means the viewer doesn't need a running
IPython kernel to be useful and web pages and presentations generated from
the notebook will work as expected.  However, this also means there is only
one-way communication between the notebook and the viewer.

##Installation
------------

From PyPI:

    !pip install py3Dmol
    
API
---

The returned view object has the exact same API as [3Dmol.GLViewer](http://3dmol.csb.pitt.edu/doc/$3Dmol.GLViewer.html)
with the exception that functions return None.

In [ ]:
##Here write down the code to install py3Dmol
!pip install py3Dmol

In [ ]:
import py3Dmol

In [ ]:
p = py3Dmol.view(query='mmtf:1ycr')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'within':{'distance': 5, 'sel':{'resi':21}}},{'stick':{'colorscheme':'blue'}})


In [ ]:
xyz = '''4
* (null), Energy   -1000.0000000
N     0.000005    0.019779   -0.000003   -0.157114    0.000052   -0.012746
H     0.931955   -0.364989    0.000003    1.507100   -0.601158   -0.004108
H    -0.465975   -0.364992    0.807088    0.283368    0.257996   -0.583024
H    -0.465979   -0.364991   -0.807088    0.392764    0.342436    0.764260
'''

In [ ]:
xyzview = py3Dmol.view(width=400,height=400)
xyzview.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1}})
xyzview.setStyle({'stick':{}})
xyzview.setBackgroundColor('0xeeeeee')
xyzview.animate({'loop': 'backAndForth'})
xyzview.zoomTo()
xyzview.show()

Display local file.

In [ ]:
benz='''
     RDKit          3D

  6  6  0  0  0  0  0  0  0  0999 V2000
   -0.9517    0.7811   -0.6622 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.2847    1.3329   -0.3121 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.2365    0.5518    0.3512 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.9517   -0.7811    0.6644 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.2847   -1.3329    0.3144 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2365   -0.5518   -0.3489 C   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  2  0
  2  3  1  0
  3  4  2  0
  4  5  1  0
  5  6  2  0
  6  1  1  0
M  END
$$$$'''
view = py3Dmol.view()
view.addModel(benz,'sdf')
view.setStyle({'stick':{}})
view.setStyle({'model':0},{'stick':{'colorscheme':'cyanCarbon'}})
view.zoomTo()

You can create a single canvas object with multiple viewers arrayed in a grid (3Dmol.createViewerGrid).

In [ ]:
view = py3Dmol.view(query='pdb:1dc9',linked=False,viewergrid=(2,2))
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}},viewer=(0,1))
view.setStyle({'stick':{'colorscheme':'greenCarbon'}},viewer=(1,0))
view.setStyle({'cartoon':{'color':'spectrum'}},viewer=(1,1))
view.removeAllModels(viewer=(0,0))
view.addModel(benz,'sdf',viewer=(0,0))
view.setStyle({'stick':{}},viewer=(0,0))
view.zoomTo(viewer=(0,0))
view.render()

In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
chA = {'chain':'A'}
chB = {'chain':'B'}
view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, chA)
view.setStyle(chB,{'stick':{}})
view.show()

In [ ]:
view = py3Dmol.view(query='pdb:5ire',options={'doAssembly':True})
view.setStyle({'cartoon':{'color':'spectrum'}})
view.show()

Color by temperature factors

In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
view.setStyle({'cartoon': {'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}})

Generate an inline image of what is currently in the viewer (all white if the structure hasn't loaded yet).

In [ ]:
png = view.png()
png

In [ ]:
import requests, base64
r = requests.get('https://mmtf.rcsb.org/v1.0/full/5lgo')
view = py3Dmol.view()
view.addModel(base64.b64encode(r.content).decode(),'mmtf')
view.addUnitCell()
view.zoomTo()

In [ ]:
!pip install py3Dmol


In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import py3Dmol

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

smi = 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br'
conf = smi2conf(smi)
viewer = MolTo3DView(conf, size=(600, 300), style='sphere')
viewer.show()

In [ ]:
viewer = MolTo3DView(conf, size=(600, 300), style='sphere')
viewer.show()

In [ ]:
#!pip install ipywidgets
%reload_ext autoreload
%autoreload 2
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

smis = [ 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br',
    'CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O',
    'Cc1c(COc2cc(OCc3cccc(c3)C#N)c(CN3C[C@H](O)C[C@H]3C(O)=O)cc2Cl)cccc1-c1ccc2OCCOc2c1',
    'CCCCC(=O)NCCCCC(=O)NCCCCCC(=O)[O-]',
    "CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O"]
    
confs = [smi2conf(s) for s in smis]

def conf_viewer(idx):
    mol = confs[idx]
    return MolTo3DView(mol).show()

interact(conf_viewer, idx=ipywidgets.IntSlider(min=0,max=4-1, step=1))

In [ ]:
%reload_ext autoreload
%autoreload 2

from ipywidgets import interact,fixed,IntSlider,interact_manual
import ipywidgets

smis = [ 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br',
    'CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O',
    'Cc1c(COc2cc(OCc3cccc(c3)C#N)c(CN3C[C@H](O)C[C@H]3C(O)=O)cc2Cl)cccc1-c1ccc2OCCOc2c1',
    'CCCCC(=O)NCCCCC(=O)NCCCCCC(=O)[O-]',
    "CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O"]

confs = [smi2conf(s) for s in smis]

def style_selector(idx, s):
    conf = confs[idx]
    return MolTo3DView(conf, style=s).show()

